### Imports

In [1]:
import numpy as np
from PIL import Image, ImageDraw

### Encoding a message vector using Reed-Solomon codes

#### Code parameters

In [3]:
F = 11 # Galois Field F11
n = 9 # Size of codeword (the encoded message)
k = 5 # Size of message (the original message)

In [6]:
msg = np.array([2, 10, 3, 7, 5])
# msg = np.array([1, 2, 3, 4])
print("Message: ", msg)

Message:  [ 2 10  3  7  5]


In [11]:
eval_points = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])

encoded_msg = np.zeros(n)
for i in range(n):
    x = eval_points[i]
    # encoded_msg[i] = (msg[0] + msg[1] * x + msg[2] * x**2 + msg[3] * pow(x, 3) + msg[4] * pow(x, 4)) % F
    encoded_msg[i] = (msg[0] + msg[1] * x + msg[2] * x**2 + msg[3] * pow(x, 3)) % F

print("Encoded Message: ", encoded_msg)

# bin_msg = np.zeros(n, dtype=np.int64)
# for i in range(n):
#     bin_msg[i] = np.binary_repr(int(encoded_msg[i]), width=4)
#     print("%04)

bin_msg = []
for i in range(n):
    bin_msg.append('{0:04b}'.format(int(encoded_msg[i])))
bin_msg = np.array(bin_msg)

print(bin_msg)

binary_string = "".join(bin_msg)
print(binary_string)

Encoded Message:  [ 0.  2.  6. 10.  1. 10.  2.  8.  4.]
['0000' '0010' '0110' '1010' '0001' '1010' '0010' '1000' '0100']
000000100110101000011010001010000100


In [66]:
width, height = len(binary_string), 100
img = Image.new("RGB", (width, height), "white")

draw = ImageDraw.Draw(img)
for i, b in enumerate(binary_string):
    x0, y0 = i, 0
    x1, y1 = i + 1, height
    if b == "1":
        draw.rectangle([x0, y0, x1, y1], fill="black")
    else:
        draw.rectangle([x0, y0, x1, y1], fill="white")

img.show()

wine: invalid directory "/home/gokul/.wine" in WINEPREFIX: not an absolute path
/bin/xdg-open: line 880: x-www-browser: command not found


### Introducing errors

In [52]:
received_vector = encoded_msg.copy()
received_vector[1] = 1
received_vector[2] = 2
received_vector

array([ 0.,  1.,  2., 10.,  1., 10.,  2.,  8.,  4.])

### Decoding

#### 1. Calculating the syndromes

In [9]:
def mod_inverse(a, m):
    """
    Compute the modular inverse of a modulo m using the extended Euclidean algorithm.
    """
    a = a % m
    for x in range(1, m):
        if (a*x) % m == 1:
            return x
    raise ValueError(f"{a} has no inverse modulo {m}")


In [18]:
def calc_syndromes(received_vector, num_errors):
    """
    Calculate the syndromes of the received vector.
    """
    syndromes = np.zeros(num_errors)
    for i in range(num_errors):
        syndromes[i] = received_vector[i+1]
        for j in range(1, k+1):
            syndromes[i] += received_vector[i+1+j] * pow(eval_points[i+1], j)
        syndromes[i] = syndromes[i] % F
    return syndromes

In [62]:
syndromes = calc_syndromes(received_vector, 2)
syndromes

array([2., 8.])

In [63]:
# range(len(syndromes))
syndromes[-1]

8.0

In [48]:
def get_error_locator_poly(received_msg, syndromes):
    # # Step 1: Compute syndromes
    # syndromes = []
    # for i in range(num_errors):
    #     s_i = 0
    #     for j in range(len(received_msg)):
    #         s_i += received_msg[j] * pow(j+1, i, F)
    #     syndromes.append(s_i % F)

    # Step 2: Find error locator polynomial using Berlekamp-Massey algorithm
    b = [1]
    c = [1]
    L = 0
    m = -1
    for i in range(len(syndromes)):
        d = syndromes[i]
        for j in range(1, L+1):
            d += b[j] * syndromes[i-j]
        if d == 0:
            continue
        if L > m:
            c = b.copy()
            m = L
        factor = F - pow(int(d * mod_inverse(b[L], F)), 1, F)
        while len(b) < len(c) + i - m:
            b.append(0)
        for j in range(len(c)):
            b[i-m+j] += factor * c[j]
            b[i-m+j] %= F
        L = i+1
    
    # Reverse the coefficients of the error locator polynomial to get the correct order
    error_locator_poly = b[::-1]

    return error_locator_poly


In [65]:
error_locator_poly = get_error_locator_poly(received_vector, syndromes)
error_locator_poly

IndexError: list index out of range

In [26]:
alpha = 2

In [27]:
def evaluate_polynomial(poly, x):
    y = 0
    for i in range(len(poly)):
        y = (y * x + poly[-(i+1)]) % F
    return y

In [32]:
def find_error_positions(error_locator_poly):
    """
    Find the error positions using the error locator polynomial.
    """
    # error_positions = []
    # for i in range(1, n+1):
    #     if pow(i, len(error_locator_poly)-1, n) == 0:
    #         error_positions.append(i)
    # return error_positions
    error_positions = []
    for i in range(1, F+1):
        if evaluate_polynomial(error_locator_poly, pow(alpha, i, F)) == 0:
            error_positions.append(F-i)
    
    return error_positions

In [50]:
error_pos = find_error_positions(error_locator_poly)
error_pos

[]

In [35]:
def find_error_values(error_positions, received_vector):
    error_magnitudes = []
    for i in range(len(error_positions)):
        omega_i = 1
        for j in range(len(error_positions)):
            if i != j:
                omega_i *= (error_positions[i] - error_positions[j]) % F
                omega_i %= F
        omega_i = mod_inverse(omega_i, F)
        error_magnitude_i = 0
        for j in range(len(received_vector)):
            error_magnitude_i += received_vector[j] * evaluate_polynomial([1], pow(alpha, (F-error_positions[i])*j, F))
            error_magnitude_i %= F
        error_magnitude_i *= omega_i
        error_magnitude_i %= F
        error_magnitudes.append(error_magnitude_i)

    return error_magnitudes

In [39]:
error_values = find_error_values(error_pos, received_vector)

In [40]:
def correct_errors(received_vector, error_positions, error_values):
    # for i in range(len(error_positions)):
    #     received_vector[error_positions[i]] += error_values[i]
    #     received_vector[error_positions[i]] %= F
    for i in range(len(error_positions)):
        received_vector[F-error_positions[i]-1] -= find_error_values[i]
        received_vector[F-error_positions[i]-1] %= F

    # Return the corrected message
    return received_vector

In [41]:
correct_errors(received_vector, error_pos, error_values)

array([ 1.,  2.,  6., 10.,  1., 10.,  2.,  8.,  4.])